In [2]:
import ee
import geemap

In [9]:
Map = geemap.Map(center=[-11.77,-45.76], zoom=8)
Map

Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [40]:
def clusterization(image):

          
    # Load a pre-computed Landsat composite for input.
    input = image.clip(geometry).select("EVI");
  
    #Make the training dataset.
    training = input.sample(**{
    'region': geometry,
    'scale': 250,
    'numPixels': 5000
    });

    #Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(5).train(training)
    #Cluster the input using the trained clusterer.
    result = input.cluster(clusterer)
 
    #Combine the mean and standard deviation reducers.
    reducers = ee.Reducer.mean()

    #Inicialização Lista de EVI dos clustes
    lista = ee.List([])

    #Iteração sobre os 5 Clusters para calcular a estatística de cada região
    for i in range(5):

        #Cálculo da geometria
        geom = result.select("cluster").eq(i).selfMask().reduceToVectors(**{
        'scale':250,
        });



        #Reducer para cálculo do EVI médio de cada cluster
        stats = image.reduceRegion(**{
        'reducer': reducers,
        'geometry':geom.geometry(),
        'scale':250,
        'bestEffort':True,
        'maxPixels':1000000000,
        });

        #Armarzena o EVI médio de cada clúster em uma lista
        lista = lista.add(stats.get("EVI"));


    #Cópia da Lista para ordenar os 2 maiores EVIs
    lista2 = lista;


    #Ordenar a lista para pegar os 2 maiores valores de EVI médio nos clusters
    #Agora nossos clustests de interesse estão nas posições 3 e 4 da lista 2
    lista2 = lista2.sort()

    #Pegar as chaves dos clusteres de interesse na lista original
    chaves1 = lista.indexOf(lista2.get(3))
    chaves2 = lista.indexOf(lista2.get(4))


    #Cálculo da Geometria final

    geom1 = result.select("cluster").eq(chaves1).selfMask().reduceToVectors(**{
    'scale':250,
    });

    geom2 = result.select("cluster").eq(chaves2).selfMask().reduceToVectors(**{
    'scale':250,
    });

    #Geom é a feature collection de nossos clusters de interesse
    geom = geom1.merge(geom2);



    stats = image.reduceRegion(**{
    'reducer': reducers,
    'geometry':geom.geometry(),
    'scale':250,
    'bestEffort':True,
    'maxPixels':1000000000,
    });


    return image.set('EVI_Cluster', stats.get("EVI"))
    

In [11]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

Map(bottom=35226.0, center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [22]:
reducers = ee.Reducer.mean()


# Reducer para cálculo do EVI médio de cada cluster
stats = lst.reduceRegion(**{
  'reducer': reducers,
  'geometry':region,
  'scale':250,
});

In [24]:
print(stats.getInfo())

{'NDVI': 8108.787373042909}


In [85]:
    #Filtrar a territorialidade 
    mcollection = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate('2020-01-01', '2021-04-30').filter(ee.Filter.calendarRange(1, 4,'month')).map(clusterization)
    #4 principais meses de safra
    #filter start and end date
    
    

In [1]:
import pandas as pd

In [86]:
a = mcollection.aggregate_array('EVI_Cluster')

In [87]:
x = a.getInfo()

In [88]:
df = pd.DataFrame({'lin':x})

In [89]:
df

,lin
0,7385.276813
1,8495.536356
2,8148.692939
3,8456.677720
4,7373.441144
5,7075.421447
6,6314.883616
7,6648.418022
8,7376.757254
9,8289.355512


In [2]:
import requests

#API Estação INMET
#Recuperar dados da estação mais próxima ao geocode passado como parâmetro:
#https://apiprevmet3.inmet.gov.br/estacao/proxima/<geocode>



#table = []



import json

geocode = '2919553'

base_site = 'https://apiprevmet3.inmet.gov.br/estacao/proxima/' + geocode

r = requests.get(base_site)

json = r.json()

dt = pd.DataFrame(json['dados'],index=[0])

estacao = dt['CD_ESTACAO'].values[0]

base_site2 = 'https://apitempo.inmet.gov.br/estacao/diaria/2000-01-01/2020-12-31/' + estacao

r = requests.get(base_site2)

json = r.json()

dt2 = pd.DataFrame(json)


In [3]:
dt2

,UMID_MED,DT_MEDICAO,DC_NOME,UMID_MIN,TEMP_MED,CHUVA,VL_LATITUDE,TEMP_MIN,TEMP_MAX,UF,VEL_VENTO_MED,CD_ESTACAO,VL_LONGITUDE
0,None,2000-01-01,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
1,None,2000-01-02,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
2,None,2000-01-03,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
3,None,2000-01-04,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
4,None,2000-01-05,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7666,74.2,2020-12-27,BARREIRAS,47,26.4,None,-12.12472221,21.2,32.2,BA,1.3,A402,-45.02694443
7667,73.3,2020-12-28,BARREIRAS,44,25.8,None,-12.12472221,19.5,32.3,BA,0.7,A402,-45.02694443
7668,72,2020-12-29,BARREIRAS,39,26.3,None,-12.12472221,20.3,33.6,BA,0.8,A402,-45.02694443
7669,66,2020-12-30,BARREIRAS,39,26.8,None,-12.12472221,20,33.9,BA,1,A402,-45.02694443


,DC_NOME,VL_LATITUDE,UF,CD_ESTACAO,VL_LONGITUDE
DT_MEDICAO,,,,,
January,BARREIRASBARREIRASBARREIRASBARREIRASBARREIRASB...,-12.12472221-12.12472221-12.12472221-12.124722...,BABABABABABABABABABABABABABABABABABABABABABABA...,A402A402A402A402A402A402A402A402A402A402A402A4...,-45.02694443-45.02694443-45.02694443-45.026944...
February,BARREIRASBARREIRASBARREIRASBARREIRASBARREIRASB...,-12.12472221-12.12472221-12.12472221-12.124722...,BABABABABABABABABABABABABABABABABABABABABABABA...,A402A402A402A402A402A402A402A402A402A402A402A4...,-45.02694443-45.02694443-45.02694443-45.026944...
March,BARREIRASBARREIRASBARREIRASBARREIRASBARREIRASB...,-12.12472221-12.12472221-12.12472221-12.124722...,BABABABABABABABABABABABABABABABABABABABABABABA...,A402A402A402A402A402A402A402A402A402A402A402A4...,-45.02694443-45.02694443-45.02694443-45.026944...
April,BARREIRASBARREIRASBARREIRASBARREIRASBARREIRASB...,-12.12472221-12.12472221-12.12472221-12.124722...,BABABABABABABABABABABABABABABABABABABABABABABA...,A402A402A402A402A402A402A402A402A402A402A402A4...,-45.02694443-45.02694443-45.02694443-45.026944...
May,BARREIRASBARREIRASBARREIRASBARREIRASBARREIRASB...,-12.12472221-12.12472221-12.12472221-12.124722...,BABABABABABABABABABABABABABABABABABABABABABABA...,A402A402A402A402A402A402A402A402A402A402A402A4...,-45.02694443-45.02694443-45.02694443-45.026944...
...,...,...,...,...,...
August,BARREIRASBARREIRASBARREIRASBARREIRASBARREIRASB...,-12.12472221-12.12472221-12.12472221-12.124722...,BABABABABABABABABABABABABABABABABABABABABABABA...,A402A402A402A402A402A402A402A402A402A402A402A4...,-45.02694443-45.02694443-45.02694443-45.026944...
September,BARREIRASBARREIRASBARREIRASBARREIRASBARREIRASB...,-12.12472221-12.12472221-12.12472221-12.124722...,BABABABABABABABABABABABABABABABABABABABABABABA...,A402A402A402A402A402A402A402A402A402A402A402A4...,-45.02694443-45.02694443-45.02694443-45.026944...
October,BARREIRASBARREIRASBARREIRASBARREIRASBARREIRASB...,-12.12472221-12.12472221-12.12472221-12.124722...,BABABABABABABABABABABABABABABABABABABABABABABA...,A402A402A402A402A402A402A402A402A402A402A402A4...,-45.02694443-45.02694443-45.02694443-45.026944...


In [25]:
mask = (dt2['DT_MEDICAO'] >= '2010-11-1') & (dt2['DT_MEDICAO'] <= '2011-4-30')
dt3 = dt2.loc[mask]
dt3[['CHUVA']] = dt3[['CHUVA']].astype(float)

C:\Users\Guga\miniconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [26]:
#Ensure df['date'] is a Series with dtype datetime64[ns]:
dt3['DT_MEDICAO'] = pd.to_datetime(dt3['DT_MEDICAO']) 

dg = dt3.groupby(pd.Grouper(key='DT_MEDICAO', freq='1M')).mean() # groupby each 1 month
dg.index = dg.index.strftime('%B')
dg

<ipython-input-26-0d6f7558dcae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt3['DT_MEDICAO'] = pd.to_datetime(dt3['DT_MEDICAO'])


,CHUVA
DT_MEDICAO,
November,6.060000
December,7.180645
January,7.754839
February,4.985714
March,6.482759
April,1.013333


In [29]:
lista = []
lista.append(dg)

In [30]:
lista

[               CHUVA
 DT_MEDICAO          
 November    6.060000
 December    7.180645
 January     7.754839
 February    4.985714
 March       6.482759
 April       1.013333]